In [ ]:
# Python script to create a GitHub issue using the GitHub API
#
# Prerequisites:
# 1. A GitHub Personal Access Token (PAT) with 'repo' scope.
#    - You can create one here: https://github.com/settings/tokens
# 2. The 'requests' library installed.
#    - You can install it by running: pip install requests

import requests
import json

# --- CONFIGURATION ---
# Replace these placeholders with your actual data

# Your GitHub Personal Access Token. Keep this secret!
# It's best practice to load this from an environment variable or a secure vault.
GITHUB_TOKEN= 

# The owner of the repository (e.g., 'octocat')
REPO_OWNER = "aiopenscale"

# The name of the repository (e.g., 'Hello-World')
REPO_NAME = "aifs-documentation"

# Your GitHub Enterprise hostname. For example: 'github.yourcompany.com'
# If you are using public github.com, leave this as None.
GITHUB_HOSTNAME = "github.ibm.com" # e.g., "github.ibm.com"


def create_github_issue(title, body=None, assignees=None, labels=None):
    """
    Creates a new issue on a specified GitHub repository.

    Args:
        title (str): The title of the issue.
        body (str, optional): The contents of the issue. Defaults to None.
        assignees (list, optional): A list of GitHub usernames to assign to the issue.
                                    Defaults to None.
        labels (list, optional): A list of label names to add to the issue.
                                 Defaults to None.

    Returns:
        bool: True if the issue was created successfully, False otherwise.
    """
    # Construct the API URL based on whether a custom hostname is provided
    if GITHUB_HOSTNAME:
        # URL for GitHub Enterprise
        url = f"https://{GITHUB_HOSTNAME}/api/v3/repos/{REPO_OWNER}/{REPO_NAME}/issues"
    else:
        # URL for public GitHub
        url = f"https://api.github.com/repos/{REPO_OWNER}/{REPO_NAME}/issues"

    # Set up the headers for the API request, including authentication
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json",
    }

    # Create the data payload for the new issue
    data = {
        "title": title,
        "body": body,
        "assignees": assignees if assignees else [],
        "labels": labels if labels else []
    }

    print(f"Sending request to create issue: '{title}' at {url}")

    try:
        # Make the POST request to the GitHub API
        response = requests.post(url, data=json.dumps(data), headers=headers)

        # Raise an exception for bad status codes (4xx or 5xx)
        response.raise_for_status()

        # If the request was successful (status code 201 Created)
        response_data = response.json()
        print(f"✅ Successfully created issue #{response_data['number']}.")
        print(f"   View it here: {response_data['html_url']}")
        return True

    except requests.exceptions.RequestException as e:
        # Handle potential errors like network issues or invalid API responses
        print(f"❌ Error creating issue: {e}")
        # Print the response content for more detailed error info if available
        if 'response' in locals() and response.content:
            print(f"   Response from server: {response.text}")
        return False


# --- SCRIPT EXECUTION ---
if __name__ == "__main__":
    # Example usage of the function
    # Define the details for the new issue you want to create
    issue_title = "Factsheets (system facts and clones)"
    # issue_body = "This is a test issue created by a Python script. ✨\n\nWe can even include **markdown**!"
    # issue_assignees = []  # Optional: e.g., ["username1", "username2"]
    # issue_labels = ["bug", "automation"] # Optional: e.g., ["bug", "help wanted"]

    # Check if the configuration variables have been changed from placeholders
    if GITHUB_TOKEN == "YOUR_PERSONAL_ACCESS_TOKEN" or \
       REPO_OWNER == "YOUR_REPOSITORY_OWNER" or \
       REPO_NAME == "YOUR_REPOSITORY_NAME":
        print("⚠️  Please update the configuration variables (GITHUB_TOKEN, REPO_OWNER, REPO_NAME) at the top of the script.")
    else:
        # Call the function to create the issue
        create_github_issue(issue_title)

Sending request to create issue: 'Factsheets (system facts and clones)' at https://github.ibm.com/api/v3/repos/aiopenscale/aifs-documentation/issues
✅ Successfully created issue #23.
   View it here: https://github.ibm.com/aiopenscale/aifs-documentation/issues/23


In [3]:
def get_api_base_url():
    """Constructs the base API URL."""
    if GITHUB_HOSTNAME:
        return f"https://{GITHUB_HOSTNAME}/api/v3"
    else:
        return "https://api.github.com"

def list_accessible_repos():
    """
    Lists all repositories accessible to the authenticated user.
    This includes public, private, and internal repositories.
    Handles pagination to retrieve all results.

    Returns:
        list: A list of repository full names (e.g., 'owner/repo'), or None if an error occurs.
    """
    repos_url = f"{get_api_base_url()}/user/repos"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json",
    }
    params = {
        "per_page": 100, # Request the maximum number of items per page
    }
    
    all_repos = []
    url = repos_url

    print("Fetching accessible repositories...")

    try:
        while url:
            response = requests.get(url, headers=headers, params=params)
            response.raise_for_status()
            
            # Add the full names of the repos from the current page to our list
            repos_page = response.json()
            all_repos.extend([repo['full_name'] for repo in repos_page])
            
            # Check for the 'next' page link in the headers
            if 'next' in response.links:
                url = response.links['next']['url']
                # Params are included in the next URL, so we clear them for subsequent requests
                params = None 
            else:
                url = None # No more pages

        print(f"✅ Found {len(all_repos)} repositories.")
        return all_repos

    except requests.exceptions.RequestException as e:
        print(f"❌ Error listing repositories: {e}")
        if 'response' in locals() and response.content:
            print(f"   Response from server: {response.text}")
        return None


In [5]:
repo_list = list_accessible_repos()

Fetching accessible repositories...
✅ Found 313 repositories.


In [12]:
[i for i in repo_list if "aiopenscale" in i]

['aiopenscale/ai-ops-squad-tests',
 'aiopenscale/aiopenscale-broker',
 'aiopenscale/aiopenscale-deployment',
 'aiopenscale/aiopenscale-dev',
 'aiopenscale/aiopenscale-monitoring',
 'aiopenscale/aiopenscale-newrelic',
 'aiopenscale/aiopenscale-scans',
 'aiopenscale/aios-cloudant-backup',
 'aiopenscale/aios-composite-service',
 'aiopenscale/aios-configmaps',
 'aiopenscale/aios-custom-engine',
 'aiopenscale/aios-event-client-python',
 'aiopenscale/aios-fvt',
 'aiopenscale/aios-java-ubi-minimal-base',
 'aiopenscale/aios-kpi-manager-service',
 'aiopenscale/aios-liberty-base',
 'aiopenscale/aios-messages',
 'aiopenscale/aios-monitors',
 'aiopenscale/aios-node-base',
 'aiopenscale/aios-operations',
 'aiopenscale/aios-orch-common-clients',
 'aiopenscale/aios-orch-common-repos',
 'aiopenscale/aios-orch-common-utils',
 'aiopenscale/aios-payload-logging-spark',
 'aiopenscale/aios-performance-test',
 'aiopenscale/aios-python-base',
 'aiopenscale/aios-python-ubi-minimal-base',
 'aiopenscale/aios-sc

In [19]:
a = """Use cases <- anyone
Associating workspaces <- Shreya
Model Tracking mechanism  <- Femi
Models Promotion mechanism <-
Factsheet settings and the default inventory  <- Puneet
Attachments <- Ranjana
Reports and Report templates  <- Ranjana
Custom facts <- Ranjana
External models  <- Jinu/Femi
Openpages integration <- Deirdre
Multicluster support  <- Ranjana
Evaluation studio <- ShreyaFactsheets (system facts and clones) Jinu
Use cases <- anyone
Associating workspaces <- Shreya
Model Tracking mechanism  <- Femi
Models Promotion mechanism <-
Factsheet settings and the default inventory  <- Puneet
Attachments <- Ranjana
Reports and Report templates  <- Ranjana
Custom facts <- Ranjana
External models  <- Jinu/Femi
Openpages integration <- Deirdre
Multicluster support  <- Ranjana
Evaluation studio <- Shreya"""

In [21]:
a = a.split('\n')

In [25]:
temp = [i.split("<-")[0].strip() for i in a]

In [29]:
for i in tqdm(temp):
    body="""
    The expectation is to create a documentation page that any developer can use to understand and work on the above-mentioned feature.
    The document should contain information on the asset types used, the assets created, and the events that cause the asset to be modified. (if applicable)
    """
    create_github_issue(title=i, body=body)

  0%|                                                                                                                                                        | 0/24 [00:00<?, ?it/s]

Sending request to create issue: 'Use cases' at https://github.ibm.com/api/v3/repos/aiopenscale/aifs-documentation/issues


  4%|██████                                                                                                                                          | 1/24 [00:01<00:36,  1.57s/it]

✅ Successfully created issue #24.
   View it here: https://github.ibm.com/aiopenscale/aifs-documentation/issues/24
Sending request to create issue: 'Associating workspaces' at https://github.ibm.com/api/v3/repos/aiopenscale/aifs-documentation/issues


  8%|████████████                                                                                                                                    | 2/24 [00:03<00:36,  1.64s/it]

...

✅ Successfully created issue #45.
   View it here: https://github.ibm.com/aiopenscale/aifs-documentation/issues/45
Sending request to create issue: 'Multicluster support' at https://github.ibm.com/api/v3/repos/aiopenscale/aifs-documentation/issues


 96%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████      | 23/24 [00:40<00:01,  1.87s/it]

✅ Successfully created issue #46.
   View it here: https://github.ibm.com/aiopenscale/aifs-documentation/issues/46
Sending request to create issue: 'Evaluation studio' at https://github.ibm.com/api/v3/repos/aiopenscale/aifs-documentation/issues


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:42<00:00,  1.77s/it]

✅ Successfully created issue #47.
   View it here: https://github.ibm.com/aiopenscale/aifs-documentation/issues/47


In [27]:
!pip install tqdm

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)


In [28]:
from tqdm import tqdm

In [ ]:
For prompts/detached PTA's in projects issue tracker is: https://github.ibm.com/dap/dap-planning/issues

For wml  and other models in projects tracker is: https://github.ibm.com/NGP-TWC/ml-planning/issues

For prompts/detached PTA's , wml and other models in spaces/deployments issue tracker is: https://github.ibm.com/NGP-TWC/ml-planning/issues

For issues related to OpenScale screen: https://github.ibm.com/aiopenscale/tracker/issues 

In [30]:
!pip install langchain langchain-openai pydantic

  Using cached pydantic-2.11.7-py3-none-any.whl.metadata (67 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.33.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.8 kB)
  Using cached typing_inspection-0.4.1-py3-none-any.whl.metadata (2.6 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached jiter-0.10.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.whl.metadata (14 kB)
  Using cached zstandard-0.23.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (3.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.3 MB/s  0:00:00
Using cached pydantic-2.11.7-py3-none-any.whl (444 kB)
Using cached pydantic_core-2.33.2-cp311-cp311-macosx_11_0_arm64.whl (1.9 MB)
Using cached jsonpatch-1.33-py2.py3-none-any.whl (12 kB)
 

In [34]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    # Prompt for the key if it's not found in the environment
    api_key = getpass.getpass("OpenAI API Key not found. Please enter it here: ")
    os.environ["OPENAI_API_KEY"] = api_key
    print("✅ OpenAI API Key has been set for this session.")
else:
    # Acknowledge that the key is already set
    print("✅ OpenAI API Key is already set.")

# You can now safely use the key, for example:
# from openai import OpenAI
# client = OpenAI() # The client automatically reads the environment variable

✅ OpenAI API Key is already set.


In [37]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# --- 1. Your Repository Data ---
repo_list = [
    {
        "url": "https://github.ibm.com/dap/dap-planning/issues",
        "description": "For issues related to prompt template assets (PTA) and detached prompt template assets (DTPA) in a project."
    },
    {
        "url": "https://github.ibm.com/NGP-TWC/ml-planning/issues",
        "description": "For issues related to Watson Machine Learning (WML) models. Also for issues related to Deployment Spaces."
    },
    {
        "url": "https://github.ibm.com/aiopenscale/tracker/issues",
        "description": "For issues related to the evaluation of a model or PTA, or any issue related to the Openscale service."
    },
]

# --- 2. Define Your Desired Output Structure ---
# We want the LLM to return the URL of the chosen repository.
class RepoChoice(BaseModel):
    repo_url: str = Field(description="The URL of the single best repository for the issue.")

# --- 3. Format the Repository List for the Prompt ---
# We'll convert the list of dicts into a clean string for the LLM.
formatted_repos = "\n---\n".join([
    f"URL: {repo['url']}\nDescription: {repo['description']}" for repo in repo_list
])

# --- 4. Create the Prompt Template ---
prompt_template = PromptTemplate.from_template(
    """You are an expert at routing software engineering issues to the correct repository.
Based on the user's issue description, choose the single most appropriate repository from the list below.

Here are the available repositories and their descriptions:
{repos}

User's Issue:
"{user_issue}"

You must choose one repository.
"""
)

# --- 5. Set up the LLM and the Chain ---
# We initialize the LLM and bind our Pydantic class to it.
# This forces the LLM to return JSON that matches the RepoChoice structure.
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm = llm.with_structured_output(RepoChoice)

# Create the full chain using LangChain Expression Language (LCEL)
chain = prompt_template | structured_llm

# --- 6. Run the Chain with Example Issues ---

print("--- Routing Examples ---")

# Example 1: Should go to dap-planning
issue_1 = "My PTA is failing to save and I'm getting a 500 error."
choice_1 = chain.invoke({"repos": formatted_repos, "user_issue": issue_1})
print(f"Issue: '{issue_1}'")
print(f"Chosen Repo URL: {choice_1.repo_url}\n")
# 

# Example 2: Should go to ml-planning
issue_2 = "The WML model deployment is stuck in the 'in progress' state in our deployment space."
choice_2 = chain.invoke({"repos": formatted_repos, "user_issue": issue_2})
print(f"Issue: '{issue_2}'")
print(f"Chosen Repo URL: {choice_2.repo_url}\n")

# Example 3: Should go to aiopenscale/tracker
issue_3 = "The Openscale service is not correctly evaluating the fairness score for my model."
choice_3 = chain.invoke({"repos": formatted_repos, "user_issue": issue_3})
print(f"Issue: '{issue_3}'")
print(f"Chosen Repo URL: {choice_3.repo_url}\n")

/Users/puneet/Documents/learn/github-issue-creator/env/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1893: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


--- Routing Examples ---
Issue: 'My PTA is failing to save and I'm getting a 500 error.'
Chosen Repo URL: https://github.ibm.com/dap/dap-planning/issues

Issue: 'The WML model deployment is stuck in the 'in progress' state in our deployment space.'
Chosen Repo URL: https://github.ibm.com/NGP-TWC/ml-planning/issues

Issue: 'The Openscale service is not correctly evaluating the fairness score for my model.'
Chosen Repo URL: https://github.ibm.com/aiopenscale/tracker/issues



In [ ]:
import os
from typing import Optional
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Ensure your API key is available in the environment
# os.environ["OPENAI_API_KEY"] = "sk-..."

# --- 1. Updated List with Display Names and Usernames ---
valid_assignees = [
    {"displayName": "Anurag", "githubUsername": "anurag-gh"},
    {"displayName": "Priya", "githubUsername": "priya-dev"},
    {"displayName": "Mike", "githubUsername": "mike-ops"},
    {"displayName": "Sara", "githubUsername": "sara-ml"},
]

# --- 2. Define the Output Structure (No change needed) ---
class Assignee(BaseModel):
    """A model to hold the identified assignee's GitHub username."""
    github_username: Optional[str] = Field(
        default=None,
        description="The official GitHub username of the person mentioned."
    )

# --- 3. Set up the LLM and Chain ---
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm = llm.with_structured_output(Assignee)

# Updated prompt to handle both name types
prompt_template = PromptTemplate.from_template(
    """You are an expert at parsing text to identify an issue's assignee.
Analyze the user's issue description below.

Your task is to identify if a person is mentioned and map them to their official GitHub username.
- You MUST ONLY choose from the following list of valid users.
- A user can be mentioned by their display name or their GitHub username.
- Your final output MUST be their official `githubUsername`.
- If no one from the list is mentioned, or if it's unclear, you must return a null value.

**Valid Users:**
{valid_users}

**User's Issue:**
"{user_issue}"
"""
)

# Create the full chain
chain = prompt_template | structured_llm

# --- 4. Run the Chain with New Examples ---

print("--- Assignee Detection Examples ---")
# Format the rich list of users for the prompt
valid_users_str = "\n".join(
    [f"- Name: {user['displayName']}, Username: {user['githubUsername']}" for user in valid_assignees]
)

# Example 1: A display name is used
issue_1 = "The new machine learning model isn't loading in the testing environment. Can Sara please take a look at this?"
result_1 = chain.invoke({"valid_users": valid_users_str, "user_issue": issue_1})
print(f"Issue: '{issue_1}'")
print(f"Identified Assignee: {result_1.github_username}\n")

# Example 2: A GitHub username is used
issue_2 = "The database migration script failed. This seems like a backend task for priya-dev."
result_2 = chain.invoke({"valid_users": valid_users_str, "user_issue": issue_2})
print(f"Issue: '{issue_2}'")
print(f"Identified Assignee: {result_2.github_username}\n")

# Example 3: No valid user is mentioned
issue_3 = "The main website is down! We need all hands on deck."
result_3 = chain.invoke({"valid_users": valid_users_str, "user_issue": issue_3})
print(f"Issue: '{issue_3}'")
print(f"Identified Assignee: {result_3.github_username}\n")

In [38]:
import os
import json
from typing import Optional
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# --- 1. SETUP & DATA ---
# Ensure your API key is available in the environment
# os.environ["OPENAI_API_KEY"] = "sk-..."

# Define your application's static context data
REPO_LIST = [
    {
        "url": "https://github.ibm.com/dap/dap-planning/issues",
        "description": "For issues related to prompt template assets (PTA) and detached prompt template assets (DTPA) in a project."
    },
    {
        "url": "https://github.ibm.com/NGP-TWC/ml-planning/issues",
        "description": "For issues related to Watson Machine Learning (WML) models. Also for issues related to Deployment Spaces."
    },
    {
        "url": "https://github.ibm.com/aiopenscale/tracker/issues",
        "description": "For issues related to the evaluation of a model or PTA, or any issue related to the Openscale service."
    },
]
VALID_ASSIGNEES = [
    {"displayName": "Anurag", "githubUsername": "anurag-gh"},
    {"displayName": "Priya", "githubUsername": "priya-dev"}
]

# --- 2. DEFINE THE UNIFIED OUTPUT SCHEMA ---
class FullIssueDraft(BaseModel):
    """A complete, structured draft of a Git issue."""
    repo_url: str = Field(..., description="The URL of the single best repository for the issue.")
    assignee_username: Optional[str] = Field(default=None, description="The official GitHub username of the person mentioned.")
    title: str = Field(..., description="A concise, descriptive title, prefixed with 'Bug:', 'Feature:', or 'Task:'.")
    body: str = Field(..., description="A detailed, Markdown-formatted body for the issue. For bugs, include steps to reproduce.")

# --- 3. THE INTERACTIVE DRAFTER CLASS ---
class IssueDrafter:
    def __init__(self):
        # Store the original request and the latest draft
        self.original_request: str = ""
        self.current_draft: Optional[FullIssueDraft] = None

        # Initialize the LLM. We use one LLM for both initial and refinement calls.
        llm = ChatOpenAI(model="gpt-4o", temperature=0)
        structured_llm = llm.with_structured_output(FullIssueDraft)

        # --- Chain for Initial Draft Generation ---
        initial_prompt = PromptTemplate.from_template(
            """You are an expert engineering assistant. Your task is to convert a user's raw text into a complete, structured Git issue draft.
            Analyze the user's request and fill in all fields: repo, assignee, title, and body.

            **Context - Repositories:**
            {valid_repos}

            **Context - Valid Assignees:**
            {valid_assignees}

            **User's Raw Request:**
            "{user_issue}"
            """
        )
        self.initial_chain = initial_prompt | structured_llm

        # --- Chain for Refining an Existing Draft ---
        refinement_prompt = PromptTemplate.from_template(
            """You are an AI assistant helping a user refine a Git issue draft.
            Your task is to modify the 'Current Draft' based on the 'User's Modification Request'.
            You must return a complete, updated draft in the exact same structured format.

            **Original User Request (for context):**
            {original_user_request}

            **Current Draft (in JSON format):**
            {current_draft_json}

            **User's Modification Request:**
            "{user_modification_request}"
            """
        )
        self.refinement_chain = refinement_prompt | structured_llm

        # Format context data once
        self.valid_repos_str = "\n".join([f"- {repo['url']}: {repo['description']}" for repo in REPO_LIST])
        self.valid_assignees_str = "\n".join([f"- {user['displayName']} ({user['githubUsername']})" for user in VALID_ASSIGNEES])

    def generate_initial_draft(self, user_request: str) -> FullIssueDraft:
        print("⚙️ Generating initial draft...")
        self.original_request = user_request
        self.current_draft = self.initial_chain.invoke({
            "user_issue": user_request,
            "valid_repos": self.valid_repos_str,
            "valid_assignees": self.valid_assignees_str
        })
        return self.current_draft

    def refine_draft(self, modification_request: str) -> FullIssueDraft:
        print(f"\n⚙️ Refining draft with request: '{modification_request}'...")
        if not self.current_draft:
            raise ValueError("An initial draft must be generated before it can be refined.")

        self.current_draft = self.refinement_chain.invoke({
            "original_user_request": self.original_request,
            "current_draft_json": self.current_draft.json(), # Serialize the current Pydantic object to a JSON string
            "user_modification_request": modification_request
        })
        return self.current_draft

# --- 4. DEMO OF THE WORKFLOW ---
def display_draft(draft_name: str, draft: FullIssueDraft):
    print(f"\n--- {draft_name} ---")
    print(f"Repository: {draft.repo_url}")
    print(f"Assignee:   {draft.assignee_username}")
    print(f"Title:      {draft.title}")
    print("\n--- Body ---\n" + draft.body)
    print("---------------------------------")

if __name__ == "__main__":
    drafter = IssueDrafter()

    # --- Step 1: User provides the initial description ---
    initial_description = "The PTA service is crashing when I try to save. Priya said she saw this too. It's a critical failure."
    first_draft = drafter.generate_initial_draft(initial_description)
    display_draft("DRAFT 1: INITIAL GENERATION", first_draft)

   

/Users/puneet/Documents/learn/github-issue-creator/env/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:1893: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


⚙️ Generating initial draft...

--- DRAFT 1: INITIAL GENERATION ---
Repository: https://github.ibm.com/dap/dap-planning/issues
Assignee:   priya-dev
Title:      Bug: PTA Service Crashes on Save

--- Body ---
### Description
The Prompt Template Assets (PTA) service is experiencing a critical failure when attempting to save. This issue has been confirmed by multiple users.

### Steps to Reproduce
1. Open the PTA service.
2. Attempt to save any changes.
3. Observe the service crashing.

### Expected Behavior
The service should save changes without crashing.

### Actual Behavior
The service crashes upon attempting to save.

### Additional Information
- This issue has been observed by Priya as well.
- It is considered a critical failure and needs immediate attention.
---------------------------------


In [ ]:
 # --- Step 2: User wants to modify the first draft ---
    modification_1 = "This is a blocker for the release. Please make the title more urgent and add that to the body."
    second_draft = drafter.refine_draft(modification_1)
    display_draft("DRAFT 2: AFTER FIRST REFINEMENT", second_draft)

    # --- Step 3: User wants to change the assignee ---
    modification_2 = "Actually, Anurag is the expert on the PTA service. Assign it to him instead."
    third_draft = drafter.refine_draft(modification_2)
    display_draft("DRAFT 3: AFTER SECOND REFINEMENT", third_draft)

In [3]:
import os
import sys
import requests

# --- Configuration ---
# Your GitHub Enterprise Personal Access Token (PAT)
# It's recommended to set this as an environment variable for security.
TOKEN = os.getenv('GITHUB_PAT')

# The specific API endpoint you provided.
API_URL = "https://github.ibm.com/api/v3/repos/wdp-gov/tracker/collaborators"

# --- Main Script ---

if not TOKEN:
    print("Error: GITHUB_TOKEN environment variable not set.")
    print("Please set your Personal Access Token as an environment variable.")
    # Exits the script if the token is not found.
    sys.exit(1)

# Standard headers for GitHub API requests
headers = {
    'Authorization': f'token {TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}

all_collaborators = []
url = API_URL

print(f"Fetching collaborators from {url}...")

try:
    # This loop will continue as long as a 'next' page URL is provided by the API.
    while url:
        response = requests.get(url, headers=headers)
        # Raises an exception for bad responses (4xx or 5xx)
        response.raise_for_status() 

        # Add the collaborators from the current page to our master list.
        all_collaborators.extend(response.json())

        # The 'requests' library handily parses the 'Link' header.
        # We look for the link to the 'next' page. If it doesn't exist, the loop ends.
        url = response.links.get('next', {}).get('url')
        if url:
            print("Fetching next page...")

    print("\n✅ Success! All collaborators have been fetched.")
    print(f"Total number of collaborators: {len(all_collaborators)}")

    # --- Display Results ---
    print("-" * 30)
    print("Username\t | Display Name")
    print("-" * 30)
    for user in all_collaborators:
        # The display name might be None if the user hasn't set one.
        display_name = user.get('name') if user.get('name') else "Not set"
        print(f"{user['login']:<15} | {display_name}")

except requests.exceptions.HTTPError as err:
    print(f"HTTP Error occurred: {err}")
    print(f"Status Code: {err.response.status_code}")
    print(f"Response: {err.response.text}")
except requests.exceptions.RequestException as err:
    print(f"An error occurred: {err}")

Fetching collaborators from https://github.ibm.com/api/v3/repos/wdp-gov/tracker/collaborators...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
Fetching next page...
F

In [4]:
text = '''
Ajiyos Yohannan
Ajiyos
Senior Manager - WatsonX Governance Factsheet Service, Data and AI
Channel Manager


Atul kumar Karn
Atul kumar Karn


Deirdre Moran
Deirdre Moran
Software Developer


femi aluko
femi
:palm_tree:
Software Developer


Harshit Gupta
harshit.gupta4


Jaswanth Pallavolu
jaswanth.pallavolu
Software Developer, Watsonx.gov


Jinu Shaji
Jinu Shaji
:face_with_thermometer:
Software Developer - WatsonX Governance


Joel Varghese
joevar
:vacation_upcoming:
Software Developer, Watsonx.Governance


Lakshmi Narayanan
Lakshmi
:spiral_calendar_pad:
Python(Data Science) developer


Laukesh Yadav
Laukesh


Puneet U(you)
puneetudhayan


Ranjana V
Ranjana
:spiral_calendar_pad:
Software Developer


Rohit Viswanath
Rohit
Software Engineer, Watsonx Governance, IBM Data & AI


Sanjay Balamurugan
Sanjay Balamurugan


Shaik Mohammed Irfan Saeed
Irfan Saeed
Software Developer, watsonx.governance


SHASHIDHAR Y
syellare
Software Engineer, Ai Governance, Factsheet


Shreya Narayanan
shreya.n


Shruti Nimgaonkar
snimgaonkar
:face_with_thermometer:
Software Developer


Shubham Kale
Shubham Kale
Software Developer - Watsonx Governance


Suhas Gowda Harish
Suhas Gowda Harish


Sumangala Palled
Sumangala Palled'''

In [24]:
all_collaborators[69]

{'login': 'kanellos',
 'id': 4552,
 'node_id': 'MDQ6VXNlcjQ1NTI=',
 'avatar_url': 'https://avatars.github.ibm.com/u/4552?',
 'gravatar_id': '',
 'url': 'https://github.ibm.com/api/v3/users/kanellos',
 'html_url': 'https://github.ibm.com/kanellos',
 'followers_url': 'https://github.ibm.com/api/v3/users/kanellos/followers',
 'following_url': 'https://github.ibm.com/api/v3/users/kanellos/following{/other_user}',
 'gists_url': 'https://github.ibm.com/api/v3/users/kanellos/gists{/gist_id}',
 'starred_url': 'https://github.ibm.com/api/v3/users/kanellos/starred{/owner}{/repo}',
 'subscriptions_url': 'https://github.ibm.com/api/v3/users/kanellos/subscriptions',
 'organizations_url': 'https://github.ibm.com/api/v3/users/kanellos/orgs',
 'repos_url': 'https://github.ibm.com/api/v3/users/kanellos/repos',
 'events_url': 'https://github.ibm.com/api/v3/users/kanellos/events{/privacy}',
 'received_events_url': 'https://github.ibm.com/api/v3/users/kanellos/received_events',
 'type': 'User',
 'user_vie

In [12]:
factsheet_team = [i.strip().split('\n')[0] for i in text.split('\n\n')]

In [40]:
import re
from tqdm import tqdm

factsheet_team_info = []

def sanitize_for_matching(text):
  """Converts text to lowercase and removes all non-alphanumeric characters."""
  return re.sub(r'[^a-z]', '', text.lower())

for name in tqdm(factsheet_team):
  temp = {"displayName": name, "githubUsername": None}
  sanitized_name = sanitize_for_matching(name)
  
  for collaborator in all_collaborators:
    sanitized_login = sanitize_for_matching(collaborator['login'])
    
    # Check if the sanitized username is part of the sanitized display name
    if sanitized_login in sanitized_name:
      temp["githubUsername"] = collaborator['login']
      # Match found, no need to check other collaborators
      break
      
  factsheet_team_info.append(temp)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 833.77it/s]


In [41]:
import json
json.dumps(factsheet_team_info)

'[{"displayName": "Ajiyos Yohannan", "githubUsername": null}, {"displayName": "Atul kumar Karn", "githubUsername": null}, {"displayName": "Deirdre Moran", "githubUsername": "Deirdre-Moran"}, {"displayName": "femi aluko", "githubUsername": "femi"}, {"displayName": "Harshit Gupta", "githubUsername": "Harshit-Gupta4"}, {"displayName": "Jaswanth Pallavolu", "githubUsername": "jaswanth-pallavolu1"}, {"displayName": "Jinu Shaji", "githubUsername": "Jinu-Shaji"}, {"displayName": "Joel Varghese", "githubUsername": null}, {"displayName": "Lakshmi Narayanan", "githubUsername": "Lakshmi-Narayanan2"}, {"displayName": "Laukesh Yadav", "githubUsername": "Laukesh-Yadav"}, {"displayName": "Puneet U(you)", "githubUsername": "puneet"}, {"displayName": "Ranjana V", "githubUsername": "Ranjana-V"}, {"displayName": "Rohit Viswanath", "githubUsername": "rohit-viswanath"}, {"displayName": "Sanjay Balamurugan", "githubUsername": "Sanjay-Balamurugan1"}, {"displayName": "Shaik Mohammed Irfan Saeed", "githubUsern

In [33]:
factsheet_team_info

[{'displayName': 'Ajiyos Yohannan', 'githubUsername': None},
 {'displayName': 'Atul kumar Karn', 'githubUsername': None},
 {'displayName': 'Deirdre Moran', 'githubUsername': 'Deirdre-Moran'},
 {'displayName': 'femi aluko', 'githubUsername': 'femi'},
 {'displayName': 'Harshit Gupta', 'githubUsername': None},
 {'displayName': 'Jaswanth Pallavolu', 'githubUsername': None},
 {'displayName': 'Jinu Shaji', 'githubUsername': 'Jinu-Shaji'},
 {'displayName': 'Joel Varghese', 'githubUsername': None},
 {'displayName': 'Lakshmi Narayanan', 'githubUsername': None},
 {'displayName': 'Laukesh Yadav', 'githubUsername': 'Laukesh-Yadav'},
 {'displayName': 'Puneet U(you)', 'githubUsername': 'puneet'},
 {'displayName': 'Ranjana V', 'githubUsername': 'Ranjana-V'},
 {'displayName': 'Rohit Viswanath', 'githubUsername': 'rohit-viswanath'},
 {'displayName': 'Sanjay Balamurugan', 'githubUsername': None},
 {'displayName': 'Shaik Mohammed Irfan Saeed',
  'githubUsername': 'Shaik-Mohammed-Irfan-Saeed'},
 {'display

In [52]:
[i['login'] for i in all_collaborators if 'joe' in i['login']]

['bentonyjoe', 'joe-veliyath', 'joel-jaison']